In [2]:
import numpy as np
import matplotlib.pyplot as plt

import codecs
import re
import os.path

In [3]:
def load_pres(fname):
    alltxts = []
    alllabs = []
    s=codecs.open(fname, 'r','utf-8') # pour régler le codage
    while True:
        txt = s.readline()
        if(len(txt))<5:
            break
        #
        lab = re.sub(r"<[0-9]*:[0-9]*:(.)>.*","\\1",txt)
        txt = re.sub(r"<[0-9]*:[0-9]*:.>(.*)","\\1",txt)
        if lab.count('M') >0:
            alllabs.append(-1)
        else: 
            alllabs.append(1)
        alltxts.append(txt)
    return alltxts,alllabs

In [4]:
path = "data/AFDpresidentutf8/corpus.tache1.learn.utf8"
alltxts,alllabs = load_pres(path)

# # transforme alllabs en 0 ou 1 au lieu de -1 ou 1
# alllabs = np.array(alllabs)
# alllabs = (1+alllabs)/2
# # set to int les labels
# alllabs = alllabs.astype(int) 
# alllabs = list(alllabs)


## Setup du modèle

In [5]:
# import outils de preprocessing
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download("punkt")
nltk.download("stopwords")

from functools import partial
from unidecode import unidecode


def preprocess(text, remove_punctuation=True, use_stemming=True, remove_stopwords=True):
    # En anglais, à priori les accents et autres caractères spéciaux ne sont très utiles
    # dans ce contexte, je normalise donc le texte
    text = text.lower()
    text = unidecode(text)

    tokens = nltk.word_tokenize(text)

    if remove_punctuation:
        tokens = [token for token in tokens if token not in string.punctuation]

    if remove_stopwords:
        stop_words = set(stopwords.words("english"))
        tokens = [token for token in tokens if token not in stop_words]

    if use_stemming:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]

    return " ".join(tokens)


[nltk_data] Downloading package punkt to /home/juldpnt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/juldpnt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# import models
from functools import partial
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

# Pour utiliser le pré-traitement défini plus haut
# MAIS avec des paramètres différents
keep_stopwords = partial(preprocess, remove_stopwords=False)

# Définit une pipeline avec différentes étapes de pré-traitement
linearSVC_model = Pipeline(
    [
        ("vect",
            CountVectorizer(
                preprocessor=preprocess,
                ngram_range=(1, 1),),
        ),
        ("tfidf", TfidfTransformer()),
        ("clf", LinearSVC(class_weight="balanced")),
    ]
)

Si tu veux itérer sur les modèles, il suffit de le faire sur les pipelines. Comme ça pas de prob de paramètres différents comme tout est encapsulé dans le pipeline.

## Evaluation du modèle

In [7]:
from imblearn.over_sampling import SMOTE
import numpy as np

# Reshape your data
alltxts_reshaped = np.array(alltxts).reshape(-1, 1)

# Instantiate SMOTE
smote = SMOTE()

# Apply SMOTE to the reshaped data
not_all_txts, not_all_labs = smote.fit_resample(alltxts_reshaped, alllabs)

ValueError: dtype='numeric' is not compatible with arrays of bytes/strings.Convert your data to numeric values explicitly instead.

In [ ]:
from sklearn.model_selection import train_test_split

# Diviser le dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(alltxts, alllabs, test_size=0.2, random_state=42)

print("Taille de l'ensemble d'entraînement :", len(train_texts))
print("Taille de l'ensemble de test :", len(test_texts))

Taille de l'ensemble d'entraînement : 45930
Taille de l'ensemble de test : 11483


In [ ]:
from sklearn.metrics import accuracy_score

print("Entraînement du modèle...")
linearSVC_model.fit(train_texts, train_labels)

print("Prédiction...")
predictions = linearSVC_model.predict(test_texts)

accuracy = accuracy_score(test_labels, predictions)

print("Accuracy du modèle :", accuracy)

Entraînement du modèle...


/home/juldpnt/micromamba/envs/data-science-general/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Prédiction...
Accuracy du modèle : 0.8415048332317339


### test avec imbalance learn

juste à changer les composants du pipeline en fonction de ce qu'on veut tester, créer différents objets pipeline et itérer dessus zoubidou

In [ ]:
from imblearn.pipeline import make_pipeline as make_pipeline_with_sampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

lr_clf = make_pipeline_with_sampler(
    CountVectorizer(preprocessor=preprocess),
    TfidfTransformer(),
    RandomOverSampler(random_state=42),
    LinearSVC(),
)

NameError: name 'CountVectorizer' is not defined

In [ ]:
lr_clf.fit(train_texts, train_labels)

/home/juldpnt/micromamba/envs/data-science-general/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('tfidftransformer', TfidfTransformer()),
                ('randomoversampler', RandomOverSampler(random_state=42)),
                ('linearsvc', LinearSVC())])

In [ ]:
predictions = lr_clf.predict(test_texts)

accuracy = accuracy_score(test_labels, predictions)
print("Accuracy du modèle :", accuracy)

Accuracy du modèle : 0.8553513890098406
